In [1]:
from __future__ import print_function
import time

start = time.time()

from collections import Counter, defaultdict
import random
import math
import sys
import argparse

import dynet as dy
import numpy as np



In [2]:

# format of files: each line is "word1 word2 ..."
train_file = "/home/burak/nn4nlp-code/data/ptb/train.txt"
test_file = "/home/burak/nn4nlp-code/data/ptb/valid.txt"

w2i = defaultdict(lambda: len(w2i))


def read(fname):
    """
    Read a file where each line is of the form "word1 word2 ..."
    Yields lists of the form [word1, word2, ...]
    """
    with open(fname, "r") as fh:
        for line in fh:
            sent = [w2i[x] for x in line.strip().split()]
            sent.append(w2i["<s>"])
            yield sent


In [3]:
train = list(read(train_file))
nwords = len(w2i)
test = list(read(test_file))
S = w2i["<s>"]
assert (nwords == len(w2i))

In [4]:
print(train[7])

[110, 111, 112, 113, 83, 79, 80, 35, 114, 81, 32, 115, 116, 117, 108, 118, 119, 101, 120, 121, 42, 122, 35, 123, 124, 64, 125, 101, 126, 64, 32, 127, 24]


In [5]:

# DyNet Starts
model = dy.ParameterCollection()
trainer = dy.AdamTrainer(model)



In [6]:

# Lookup parameters for word embeddings
EMBED_SIZE = 64
HIDDEN_SIZE = 128
WORDS_LOOKUP = model.add_lookup_parameters((nwords, EMBED_SIZE))



In [7]:
WORDS_LOOKUP.value()

array([[ 0.00611134,  0.00585754, -0.00495744, ..., -0.02162647,
        -0.01951803,  0.00704189],
       [-0.00011651, -0.02292471, -0.00679104, ...,  0.00653533,
         0.00588411,  0.00255642],
       [-0.01538066,  0.01237532, -0.00159498, ..., -0.00801592,
        -0.00016228, -0.01361238],
       ...,
       [ 0.02410182,  0.02345386, -0.01472598, ..., -0.01915732,
         0.0137163 ,  0.00248644],
       [ 0.02303668,  0.01595364,  0.00845935, ..., -0.00825492,
         0.00526582, -0.0024483 ],
       [-0.01457664, -0.00777081, -0.00282113, ...,  0.01206845,
        -0.0133062 , -0.0032909 ]])

In [8]:
# Word-level LSTM (layers=1, input=64, output=128, model)
RNN = dy.LSTMBuilder(1, EMBED_SIZE, HIDDEN_SIZE, model)

# Softmax weights/biases on top of LSTM outputs
W_exp = model.add_parameters((nwords, HIDDEN_SIZE))
b_exp = model.add_parameters(nwords)


In [7]:
WORDS_LOOKUP[S]

expression 0/0

In [9]:
def calc_lm_loss(sent):
    dy.renew_cg()
    # parameters -> expressions

    # initialize the RNN
    f_init = RNN.initial_state()

    # get the wids and masks for each step
    tot_words = len(sent)

    # start the rnn by inputting "<s>"
    s = f_init.add_input(WORDS_LOOKUP[S])

    # feed word vectors into the RNN and predict the next word
    losses = []
    for wid in sent:
        # calculate the softmax and loss
        score = W_exp * s.output() + b_exp
        loss = dy.pickneglogsoftmax(score, wid)
        losses.append(loss)
        # update the state of the RNN
        wemb = WORDS_LOOKUP[wid]
        s = s.add_input(wemb)
    return dy.esum(losses), tot_words



In [10]:
arr = [1,2,3,4]

In [11]:
calc_lm_loss(arr)[0].scalar_value()

36.810176849365234

In [8]:
# Sort training sentences in descending order and count minibatches
train_order = list(range(len(train)))

In [40]:
len(train_order)

42068

In [9]:
print("startup time: %r" % (time.time() - start))
# Perform training
start = time.time()
i = all_time = dev_time = all_tagged = this_words = this_loss = 0
for ITER in range(100):
    random.shuffle(train_order)
    for sid in train_order:
        i += 1
        if i % int(500) == 0:
            trainer.status()
            print(this_loss / this_words, file=sys.stderr)
            all_tagged += this_words
            this_loss = this_words = 0
            all_time = time.time() - start
        if i % int(10000) == 0:
            dev_start = time.time()
            dev_loss = dev_words = 0
            for sent in test:
                loss_exp, mb_words = calc_lm_loss(sent)
                dev_loss += loss_exp.scalar_value()
                dev_words += mb_words
            dev_time += time.time() - dev_start
            train_time = time.time() - start - dev_time
            print("nll=%.4f, ppl=%.4f, words=%r, time=%.4f, word_per_sec=%.4f" % (
            dev_loss / dev_words, math.exp(dev_loss / dev_words), dev_words, train_time, all_tagged / train_time))
        # train on the minibatch
        loss_exp, mb_words = calc_lm_loss(train[sid])
        this_loss += loss_exp.scalar_value()
        this_words += mb_words
        loss_exp.backward()
        trainer.update()
    print("epoch %r finished" % ITER)
    trainer.update_epoch(1.0)

startup time: 15.418585538864136


7.06296362124392
6.762869800033242
6.598255168775976
6.554767982793251
6.524601719694448
6.408691625100324


KeyboardInterrupt: 